In [46]:
import numpy as np
# from gensim.models import KeyedVectors
# # give a path of model to load function
# model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
import gensim.downloader as api
model = api.load("glove-wiki-gigaword-50")

[==================================================] 100.0% 66.0/66.0MB downloaded


In [64]:
from asyncio.windows_events import NULL
import json
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from nltk.tokenize import word_tokenize

import difflib

import pickle
from os.path import exists

import spacy

from rank_bm25 import BM25Okapi

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("merge_entities")

stemm = RSLPStemmer()

In [65]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [48]:
doc_set = {}
bm25 = NULL

In [49]:
with open('CISI/CISI.ALL') as f:
        lines = ""
        for l in f.readlines():
            lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
        lines = lines.lstrip("\n").split("\n")
doc_id = ""
doc_text = ""
for l in lines:
    if l.startswith(".I"):
        doc_id = int(l.split(" ")[1].strip())
    elif l.startswith(".X"):
        doc_set[doc_id] = doc_text.lstrip(" ")
        doc_id = ""
        doc_text = ""
    else:
        doc_text += l.strip()[3:] + " "  # The first 3 characters of a line can be ignored.

In [62]:
def preprocess_text(text):
    token_words = word_tokenize(text)
    stem_sentence = []
    for word in token_words:
        stem_sentence.append(ps.stem(word))
        stem_sentence.append(" ")
    doc = nlp("".join(stem_sentence))
    words = []
    for token in doc:
        if not token.is_punct and not token.is_space and not token.is_stop:
            tk = token.lemma_
            words.append(tk)
    return words


## function for building inverted index using bm25

def building_index(tokenized_corpus):
    bm25_in = BM25Okapi(tokenized_corpus)
    return bm25_in;

In [51]:
corpus = list(doc_set.values())

if not exists('BM25.index'):
    tokenized_corpus = [preprocess_text(doc) for doc in corpus]
    # build index for Docs
    bm25 = building_index(tokenized_corpus)
    with open('BM25.index', 'wb') as outp:
        pickle.dump(bm25, outp, pickle.HIGHEST_PROTOCOL)
else:
    with open('BM25.index', 'rb') as inp:
        bm25 = pickle.load(inp)

In [52]:
def results_from_query_new(query, bm25):
    tokenized_query = preprocess_text(query)
    scores = bm25.get_scores(tokenized_query)
    most_relevant_document = np.argsort(-scores)
    return most_relevant_document[0:100]

In [53]:
def hello_world():
    query = "Describe presently working and planned systems for publishing and printing original papers by computer, and then saving the byproduct, articles coded in data-processing form, for further use in retrieval."
    
    most_relevant_documents = results_from_query_new(query, bm25)

    return most_relevant_documents

In [54]:
m = hello_world()

In [55]:
def _embed(tokens):
    embedding = np.mean(
        np.array([model[token] for token in tokens if token in model]),
        axis=0,
    )
    unit_embedding = embedding / (embedding**2).sum()**0.5
    return unit_embedding

def rank(tokenized_query, tokenized_documents):
    query_embedding = _embed(tokenized_query) # (E,)
    document_embeddings = np.array([_embed(document) for document in tokenized_documents]) # (N, E)
    scores = document_embeddings.dot(query_embedding)
    index_rankings = np.argsort(scores)[::-1]
    return index_rankings, np.sort(scores)[::-1]

In [68]:
tokenized_corpus = [preprocess_text(doc) for doc in corpus]

In [69]:
r_doc = [tokenized_corpus[i] for i in m]

In [76]:
query = "    A new method is described to extract significant phrases in the title and the abstreact of scientific or technical documents.  The method is based upon a text structure analysis and uses a relatively small dictionary. The dictionary has been constructed based on the knowledge about concepts in the field of science or technology and some lexical knowledge.  For significant phrases and their component items may be used in different meanings among the fields.  A text analysius approach has been applied to select significant phrases as substantial and semantic information carriers of the contents of the abstract.      The results of the experiment for five sets of documents have shown that the significant phrases are effectively extracted in all cases, and the number of them for every document and the processing time is fairly satisfactory.  The information representation of the document, partly using the method, is discussed with relation to the construction of the document information retrieval system."

tokenized_query = preprocess_text(query)

In [79]:
arr1 , arr2 = rank(tokenized_query,tokenized_corpus)

In [82]:
print(m)

# new = []
# for t in arr1:
#     new.append(m[t])
# print(new)
print(arr1[0:100])

[ 419  483 1077  834  445  375  710 1390 1059 1179  460  785  211 1427
 1003  872  149  251  979  522  724  595  249 1071  957  674 1363 1193
  373  795 1190  500  205  441  663  158  227 1189  179  647  506  691
 1206 1408 1247  812  420  564 1298  324   58 1426  166  610  369 1284
   38  439  155 1251  449  970  703 1192   64  671 1151  730  489  174
   48 1263 1432  847  679  528  178 1215  561  989  248  475  571  789
 1011  134 1198  614  476  685 1108  570  863  888   25  377  533  164
    5  603]
[  70  575  315  353  552  482 1043  320  660 1225  340   27  642 1223
 1164   15  387 1053 1390 1322  563  487   78  897 1090   45  654 1447
  519   34  359  753  665  316  522  961 1072  797  114   41  655  293
 1123  703 1391  486  449 1273  656  494  264  502 1173  958  648  837
  529  724  477   61   75  175  699  861  362  783  819  441  640 1056
 1410  380 1104  515  541  510  206  570  912  346 1141    1 1018  939
  817 1019  149  701  643  794  328 1362  620  629  574  373 1161

In [66]:
k = "Hello My Name is Mohammed and I am from syria, I was born in 4/7/1998. I love to celebrate on my birthday in 7-4-1998"
new_k = preprocess_text(k)
print(new_k)

['hello', 'moham', 'syria', 'wa', 'bear', '4/7/1998', 'love', 'celebr', 'birthday', '7-4-1998']
